In [1]:
import qt
from qt import dt, np, pd

In [2]:
from eodhd_fetcher import *

In [3]:
import os
import io
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

In [4]:
EODHD_API_TOKEN = "688892463a08d2.30221935"
BASE_URL = "https://eodhd.com/api"
SESSION = requests.Session()

def make_request(endpoint, params={}, api_token=EODHD_API_TOKEN):
	"""
	Helper to make a request with error handling.
	"""
	url = f"{BASE_URL}/{endpoint}"
	params['api_token'] = api_token
	params['fmt'] = 'json'
	
	try:
		response = SESSION.get(url, params=params, timeout=10)
		if response.status_code == 200:
			try:
				return response.json()
			except ValueError:
				# Sometimes CSV is returned or invalid JSON if error
				return response.text
		elif response.status_code == 429:
			print(f"Rate limit reached for {url}. Sleeping...")
			time.sleep(60) # Wait a minute
			return make_request(endpoint, params, api_token)
		else:
			print(f"Error {response.status_code} fetching {url}: {response.text[:100]}")
			return None
	except Exception as e:
		print(f"Exception fetching {url}: {e}")
		return None

def get_all_us_stocks(api_token=EODHD_API_TOKEN):
	"""
	Get list of all US stocks, including delisted ones.
	"""
	print("Fetching all US stocks (active + delisted)...")
	# delisted=1 includes delisted tickers
	data = make_request("exchange-symbol-list/US", {"delisted": 1}, api_token=api_token)
	
	if data and isinstance(data, list):
		df = pd.DataFrame(data)
		print(f"Retrieved {len(df)} US tickers.")
		return df
	else:
		print("Failed to retrieve stock list.")
		return pd.DataFrame()

- save universe

In [5]:
t = get_all_us_stocks()
# t.to_csv('data/universe.csv', index=False)

Fetching all US stocks (active + delisted)...
Retrieved 56549 US tickers.


- 

In [6]:
t = pd.read_csv('data/universe.csv')

In [7]:
t.shape
t2 = t[t['Type'].isin(['Common Stock', 'Preferred Stock'])].reset_index(drop=True)
t2.shape

(56549, 7)

(32669, 7)

In [8]:
t2['Exchange'] .value_counts()

Exchange
NASDAQ       10395
PINK          9003
NYSE          6315
OTCGREY       2466
OTCQB          939
OTCMKTS        916
OTCBB          775
OTCCE          627
NYSE MKT       532
OTCQX          414
US             175
AMEX            39
NYSE ARCA       28
BATS            26
NMFQS           19
Name: count, dtype: int64

In [9]:
t2 = t2[t2['Exchange'].isin(['NASDAQ', 'NYSE', 'NYSE MKT', 'NYSE ARCA', 'AMEX'])].reset_index(drop=True)

In [10]:
qt.view(t2[~t2['Isin'].isna()])

Grid(columns_fit='auto', compress_data=True, css_rules_down=['.number-cell {text-align: left;width: 10;}', '.l…